### Introduction: This is the Jupyter notebook to do the following things:

1. Read slimmed PKU Tree files
2. Store the raw MC histograms to pickle files

kernel: HWW

### Import necessary modules

In [1]:
import numpy as np
import os
import awkward as ak
import matplotlib as mpl
import matplotlib.pyplot as plt
import mplhep as hep
import boost_histogram as bh
from cycler import cycler
import uproot
import pickle as pkl
import hist as hist2
from typing import Dict, List
from dataclasses import dataclass, field
from copy import deepcopy

In [2]:
YEAR = "2018"

In [3]:
MAIN_DIR = "."

plot_dir = f"{MAIN_DIR}/templates/15Jul2024"
os.makedirs(plot_dir, exist_ok=True)

### Read SlimmedTree files

In [4]:
#load the slimmedtree files using uproot

#different year available here.
# year = "2016APV"
# year = "2016"
# year = "2017"
# year = "2018"
# year = "Full-Run2"
year = YEAR

CustNanoData = {
    'data'   : f"/data/bond/lyazj/SlimmedTree/V0/{year}/Data/SlimmedTree_JetHT_*.root",
    'QCD'    : f"/data/bond/lyazj/SlimmedTree/V0/{year}/MC/SlimmedTree_QCD.root",
    'Top'    : f"/data/bond/lyazj/SlimmedTree/V0/{year}/MC/SlimmedTree_Top.root",
    'WJets'  : f"/data/bond/lyazj/SlimmedTree/V0/{year}/MC/SlimmedTree_WJets.root",
    'Rest'   : f"/data/bond/lyazj/SlimmedTree/V0/{year}/MC/SlimmedTree_Rest.root",
    'Signal' : f"/data/bond/lyazj/SlimmedTree/V0/{year}/MC/SlimmedTree_Signal.root",
}

files = { }
for typefile in CustNanoData:
    files[typefile] = uproot.lazy({CustNanoData[typefile]: "PKUTree"})

In [5]:
files

{'data': <Array [{SF_unc: 0, ... PTj_V2_c: 425}] type='41897087 * {"SF_unc": float32, "PT...'>,
 'QCD': <Array [{Mj_jesAbsolute_yearDown_b: -99, ... ] type='38349513 * {"Mj_jesAbsolute...'>,
 'Top': <Array [{b_QCDbb: -99, ... SF_unc: 0.0375}] type='7727422 * {"b_QCDbb": float32,...'>,
 'WJets': <Array [{LHEScaleWeight_8: 0.81, ... ] type='4819765 * {"LHEScaleWeight_8": floa...'>,
 'Rest': <Array [{SF_unc: 0.0162, ... Phij_max: 0.835}] type='3121448 * {"SF_unc": float6...'>,
 'Signal': <Array [{LHEScaleWeight_6: 0.98, ... ] type='4294 * {"LHEScaleWeight_6": float32...'>}

In [6]:
def add_tagger(events):
    events["probQCD"] = ak.zeros_like(events["a_Hbc"])
    qcd_list = [
        "a_QCDbb",
        "a_QCDcc",
        "a_QCDb",
        "a_QCDc",
        "a_QCDothers",
    ]
    for score in qcd_list:
        events["probQCD"] = events["probQCD"] + events[score]
    events["HbcVSQCS"] = (events["a_Hbc"])/(events["a_Hbc"] + 0.997032*events["probQCD"] + 0.002968*events["a_Hcs"])
for k in files:
    print("Add tagger of:",k)
    add_tagger(events=files[k])

Add tagger of: data
Add tagger of: QCD
Add tagger of: Top
Add tagger of: WJets
Add tagger of: Rest
Add tagger of: Signal


### some test about variables / output all the variables

In [7]:
sorted(files["WJets"].fields)

['Etaj',
 'Etaj_2',
 'Etaj_3',
 'Etaj_V2_a',
 'Etaj_V2_b',
 'Etaj_V2_c',
 'Etaj_max',
 'Etaj_mid',
 'Etaj_min',
 'HT',
 'HbcVSQCS',
 'LHEScaleWeight_0',
 'LHEScaleWeight_1',
 'LHEScaleWeight_2',
 'LHEScaleWeight_3',
 'LHEScaleWeight_4',
 'LHEScaleWeight_5',
 'LHEScaleWeight_6',
 'LHEScaleWeight_7',
 'LHEScaleWeight_8',
 'MET_et',
 'MET_et_UEdown',
 'MET_et_UEup',
 'MET_phi',
 'MET_phi_UEdown',
 'MET_phi_UEup',
 'MJJ',
 'MJJJ',
 'Mj',
 'Mj_2',
 'Mj_3',
 'Mj_V2_a',
 'Mj_V2_b',
 'Mj_V2_c',
 'Mj_corr_V2_a',
 'Mj_corr_V2_b',
 'Mj_corr_V2_c',
 'Mj_jerDown_a',
 'Mj_jerDown_b',
 'Mj_jerDown_c',
 'Mj_jerUp_a',
 'Mj_jerUp_b',
 'Mj_jerUp_c',
 'Mj_jesAbsoluteDown_a',
 'Mj_jesAbsoluteDown_b',
 'Mj_jesAbsoluteDown_c',
 'Mj_jesAbsoluteUp_a',
 'Mj_jesAbsoluteUp_b',
 'Mj_jesAbsoluteUp_c',
 'Mj_jesAbsolute_yearDown_a',
 'Mj_jesAbsolute_yearDown_b',
 'Mj_jesAbsolute_yearDown_c',
 'Mj_jesAbsolute_yearUp_a',
 'Mj_jesAbsolute_yearUp_b',
 'Mj_jesAbsolute_yearUp_c',
 'Mj_jesBBEC1Down_a',
 'Mj_jesBBEC1Down_b',

### plot setting

In [8]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import mplhep as hep
import boost_histogram as bh
from cycler import cycler

use_helvet = False ## true: use helvetica for plots, make sure the system have the font installed
if use_helvet:
    CMShelvet = hep.style.CMS
    CMShelvet['font.sans-serif'] = ['Helvetica', 'Arial']
    plt.style.use(CMShelvet)
else:
    plt.style.use(hep.style.CMS)

def flow(hist: bh.Histogram, overflow: bool=True, underflow: bool=True):
    h, var = hist.view(flow=(overflow | underflow)).value, hist.view(flow=(overflow | underflow)).variance
    if overflow: 
        # h, var also include underflow bins but in plots usually no underflow data
        # And we've filled None with -999, so we shouldn't show underflow data (mostly from filled None)
        # You have to access the overflow and underflow bins data like below:
        h[-2] += h[-1]; var[-2] += var[-1]
    if underflow:
        h[1] += h[0]; var[1] += var[0]
    if overflow or underflow:
        h, var = h[1:-1], var[1:-1]
    return h, var
    # Return the updated histogram and variance

def error_bar(h, var, type='data'):
    from scipy.interpolate import CubicSpline
    if type == 'data':
        number = h
    elif type == 'mc':  # h = k*N, var = k^2*N, std = k*sqrt(N)
        number = h**2 / var
    else:
        raise ValueError("type should be 'data' or 'mc'! ")
    center = range(11) # Number: 0-10
    up = np.array([1.84, 3.30, 4.64, 5.92, 7.16, 8.38, 9.58, 10.77, 11.95, 13.11, 14.27]) - center
    down = center - np.array([0, 0.17, 0.71, 1.37, 2.09, 2.84, 3.62, 4.42, 5.23, 6.06, 6.89])
    #cs means to create a CubicSpline object
    cs_up = CubicSpline(x=center, y=up)
    cs_down = CubicSpline(x=center, y=down)
    
    Garwood = (number>0)&(number<10)
    poison_error_bar = np.sqrt(number)
    up_error_bar = np.copy(poison_error_bar)
    down_error_bar = np.copy(poison_error_bar)
    up_error_bar[Garwood] = cs_up(number[Garwood])
    down_error_bar[Garwood] = cs_down(number[Garwood])
    if type == 'mc':
        up_error_bar *= var/h
        down_error_bar *= var/h
    up_error_bar [up_error_bar < 0 ] = 0
    down_error_bar [down_error_bar < 0 ] = 0
    return np.array([down_error_bar, up_error_bar])


# function to find the optimal region with S/sqrt(B)
# not used so far
def optimalcut(shist, bhist):
    n_bins = len(shist)
    best_lower = None
    best_upper = None
    best_s_sqrt_b = 0

    for lower in range(n_bins):
        for upper in range(lower+1, n_bins+1):
            s = np.sum(shist[lower:upper])
            b = np.sum(bhist[lower:upper])
            s_sqrt_b = s / np.sqrt(b + 1)

            if s_sqrt_b > best_s_sqrt_b:
                best_lower = lower
                best_upper = upper
                best_s_sqrt_b = s_sqrt_b

    return best_lower, best_upper, best_s_sqrt_b

def optimalcut_oneside(shist, bhist, epsilon = 0.01):
    '''
    Given the signal histogram and background histogram, 
    show the one-side cut for the variable to get best s/sqrt(b).
    Args:
        shist:signal histogram
        bhist:background histogram
        epsilon(float): epsilon to avoid numerical errs 
    '''
    n_bins = len(shist)
    best_cut = 0
    best_s_sqrt_b = 0

    for cut in range(n_bins):
        s = np.sum(shist[cut:])
        b = np.sum(bhist[cut:])
        s_sqrt_b = s / np.sqrt(b + epsilon)
        if s_sqrt_b > best_s_sqrt_b:
            best_cut = cut
            best_s_sqrt_b = s_sqrt_b
        
    return best_cut, best_s_sqrt_b

def optimalcut_mid_combine(shist1, shist2, bhist, epsilon = 1):
    '''
    Given the signal histogram and background histogram, 
    show the one-side cut for the variable to get best s/sqrt(b).
    Args:
        shist:signal histogram
        bhist:background histogram
        epsilon(float): epsilon to avoid numerical errs 
    '''
    n_bins = len(shist1)
    best_cut = 0
    best_combined_sig_two_side = 0

    for cut in range(n_bins):
        s_right_side = np.sum(shist2[cut:])
        b_right_side = np.sum(bhist[cut:])
        s_left_side = np.sum(shist1[:cut])
        b_left_side = np.sum(bhist[:cut])
        s_sqrt_b_right_side = s_right_side / np.sqrt(b_right_side + epsilon)
        s_sqrt_b_left_side = s_left_side / np.sqrt(b_left_side + epsilon)
        combined_sig_two_side = np.sqrt((s_sqrt_b_right_side)**2 + (s_sqrt_b_left_side)**2)
        if combined_sig_two_side > best_combined_sig_two_side:
            best_cut = cut
            best_combined_sig_two_side = combined_sig_two_side
        
    return best_cut, best_combined_sig_two_side


### Define observable object variables

In [9]:
@dataclass
class ShapeVar:
    """Class to store attributes of a variable to make a histogram of.

    Args:
        var (str): variable name
        label (str): variable label
        bins (List[int]): bins
        reg (bool, optional): Use a regular axis or variable binning. Defaults to True.
        blind_window (List[int], optional): if blinding, set min and max values to set 0. Defaults to None.
        significance_dir (str, optional): if plotting significance, which direction to plot it in.
          See more in plotting.py:ratioHistPlot(). Options are ["left", "right", "bin"]. Defaults to "right".
    """

    var: str = None
    label: str = None
    bins: List[int] = None
    reg: bool = True #regular axis
    blind_window: List[int] = None
    significance_dir: str = "right"

    def __post_init__(self):
        # create axis used for histogramming
        if self.reg:
            self.axis = hist2.axis.Regular(*self.bins, name=self.var, label=self.label)
        else:
            self.axis = hist2.axis.Variable(self.bins, name=self.var, label=self.label)

@dataclass
class Syst:
    samples: list[str] = None
    years: list[str] = field(default_factory=lambda: years)
    label: str = None
    
def blindBins(h: hist2.Hist, blind_region: List, blind_sample: str = None, axis=0):
    """
    Blind (i.e. zero) bins in histogram ``h``.
    If ``blind_sample`` specified, only blind that sample, else blinds all.
    """
    if axis > 0:
        raise Exception("not implemented > 1D blinding yet")

    bins = h.axes[axis + 1].edges
    lv = int(np.searchsorted(bins, blind_region[0], "right"))
    rv = int(np.searchsorted(bins, blind_region[1], "left") + 1)

    if blind_sample is not None:
        data_key_index = np.where(np.array(list(h.axes[0])) == blind_sample)[0][0]
        h.view(flow=True)[data_key_index][lv:rv].value = 0
        h.view(flow=True)[data_key_index][lv:rv].variance = 0
    else:
        h.view(flow=True)[:, lv:rv].value = 0
        h.view(flow=True)[:, lv:rv].variance = 0       
shape_vars = [
    ShapeVar(
        "MH_Reco",
        r"Wcb candidate soft-drop mass [GeV]",
        [20, 30, 230],
        reg=True,
        blind_window=[50, 110],
    )
]

### Define samples we consider

In [10]:
sig_keys = [
    "Signal",
]

bkg_keys = [
    "Top",
    "WJets",
    "Rest"
]
# no QCD here, since we will use data-driven for QCD
mc_keys = sig_keys + bkg_keys

### Define weight shift list

In [14]:
years = ["2016APV", "2016", "2017", "2018"]

weight_shifts = {
    "pileup": Syst(samples=mc_keys, label="Pileup"),
    "ISRPartonShower": Syst(samples=mc_keys, label="ISR Parton Shower"),
    "FSRPartonShower": Syst(samples=mc_keys, label="FSR Parton Shower"),
    "QCDscale": Syst(samples=bkg_keys, label="QCDScale"),
    # "PDFscale": Syst(samples=bkg_keys, label="PDFscale"),
    "L1Prefiring": Syst(samples=mc_keys, label="L1Prefiring"),
    "triggerEffSF_uncorrelated" : Syst(samples=mc_keys, label="Trigger SF"),
}

### Re-organize weight information

In [24]:
samples = list(['data','QCD','Top','WJets','Rest','Signal']) #all samples we considered
years = ["2016APV", "2016", "2017", "2018"]#not used here finally
year_to_run = YEAR #not used here finally
for shift in ["down", "up"]:
    print(shift, end=':\n')
    for wshift, wsyst in weight_shifts.items():
        for wsample in wsyst.samples:
            print("  -", wsample, end=':\n')
            if wsample in samples:
                if wshift == "pileup" :
                    print("      -", wshift)
                    if shift == "up":
                        files[wsample][f"{wshift}_{shift}"] = files[wsample]["weight"] * (files[wsample]["puWeightUp"] / files[wsample]["puWeight"])
                    if shift == "down":
                        files[wsample][f"{wshift}_{shift}"] = files[wsample]["weight"] * (files[wsample]["puWeightDown"] / files[wsample]["puWeight"])
                if wshift == "ISRPartonShower" :
                    print("      -", wshift)
                    if shift == "up":
                        files[wsample][f"{wshift}_{shift}"] = files[wsample]["weight"] * files[wsample]["PSWeight_0"]
                    if shift == "down":
                        files[wsample][f"{wshift}_{shift}"] = files[wsample]["weight"] * files[wsample]["PSWeight_2"] 
                if wshift == "FSRPartonShower" :
                    print("      -", wshift)
                    if shift == "up":
                        files[wsample][f"{wshift}_{shift}"] = files[wsample]["weight"] * files[wsample]["PSWeight_1"]
                    if shift == "down":
                        files[wsample][f"{wshift}_{shift}"] = files[wsample]["weight"] * files[wsample]["PSWeight_3"] 
                if wshift == "QCDscale" :
                    print("      -", wshift)
                    if shift == "up":
                        files[wsample][f"{wshift}_{shift}"] = files[wsample]["weight"] * files[wsample]["LHEScaleWeight_8"]
                    if shift == "down":
                        files[wsample][f"{wshift}_{shift}"] = files[wsample]["weight"] * files[wsample]["LHEScaleWeight_0"] 
                if wshift == "L1Prefiring" :
                    #note that no L1Prefiring for 2018, so simply set to 1
                    print("      -", wshift)
                    if shift == "up":
                        if year_to_run == "2018":
                            files[wsample][f"{wshift}_{shift}"] = files[wsample]["weight"]
                        else:
                            files[wsample][f"{wshift}_{shift}"] = files[wsample]["weight"] * (files[wsample]["PrefireWeightUp"] / files[wsample]["PrefireWeight"])
                    if shift == "down":
                        if year_to_run == "2018":
                            files[wsample][f"{wshift}_{shift}"] = files[wsample]["weight"]
                        else:
                            files[wsample][f"{wshift}_{shift}"] = files[wsample]["weight"] * (files[wsample]["PrefireWeightDown"] / files[wsample]["PrefireWeight"])
                if wshift == "triggerEffSF_uncorrelated" :
                    print("      -", wshift)
                    if shift == "up":
                        files[wsample][f"{wshift}_{shift}"] = files[wsample]["weight"] * (1 + files[wsample]["SF_unc"])
                    if shift == "down":
                        files[wsample][f"{wshift}_{shift}"] = files[wsample]["weight"] * (1 - files[wsample]["SF_unc"])

down:
  - Signal:
      - pileup
  - Top:
      - pileup
  - WJets:
      - pileup
  - Rest:
      - pileup
  - Signal:
      - ISRPartonShower
  - Top:
      - ISRPartonShower
  - WJets:
      - ISRPartonShower
  - Rest:
      - ISRPartonShower
  - Signal:
      - FSRPartonShower
  - Top:
      - FSRPartonShower
  - WJets:
      - FSRPartonShower
  - Rest:
      - FSRPartonShower
  - Top:
      - QCDscale
  - WJets:
      - QCDscale
  - Rest:
      - QCDscale
  - Signal:
      - L1Prefiring
  - Top:
      - L1Prefiring
  - WJets:
      - L1Prefiring
  - Rest:
      - L1Prefiring
  - Signal:
      - triggerEffSF_uncorrelated
  - Top:
      - triggerEffSF_uncorrelated
  - WJets:
      - triggerEffSF_uncorrelated
  - Rest:
      - triggerEffSF_uncorrelated
up:
  - Signal:
      - pileup
  - Top:
      - pileup
  - WJets:
      - pileup
  - Rest:
      - pileup
  - Signal:
      - ISRPartonShower
  - Top:
      - ISRPartonShower
  - WJets:
      - ISRPartonShower
  - Rest:
      - ISRPart

In [22]:
for item in files:
    print(item, hex(id(files[item])))

data 0x7f1cf0811b20
QCD 0x7f1cf09992e0
Top 0x7f15cd9e2a30
WJets 0x7f15ce252520
Rest 0x7f1d75ea4250
Signal 0x7f13c8f91a30


### Define variation shift list

In [25]:
jecs = {
    "JES"     : "JES_jes",
    "JER"     : "JER",
    "JMS"     : "JMS",
    "JMR"     : "JMR",
    "Absolute": "split",
    "Absolute_year": "split",
    "BBEC1": "split",
    "BBEC1_year": "split",
    "EC2": "split",
    "EC2_year": "split",
    "FlavorQCD": "split",
    "HF": "split",
    "HF_year": "split",
    "RelativeBal": "split",
    "RelativeSample_year": "split",
}

# uncluste = {
#     "UE": "unclusteredEnergy",
# }
# no need to use MET

# pdf = {
#     "pdfscale": "pdfscale",
# }

jec_shifts = {}
for key in jecs:
    for shift in ["up", "down"]:
        if key == "JES": 
            if shift == "up"   : jec_shifts[f"{key}_{shift}"] = "Mj_jesTotalUp_a"
            if shift == "down" : jec_shifts[f"{key}_{shift}"] = "Mj_jesTotalDown_a"
        elif key == "JER": 
            if shift == "up"   : jec_shifts[f"{key}_{shift}"] = "Mj_jerUp_a"
            if shift == "down" : jec_shifts[f"{key}_{shift}"] = "Mj_jerDown_a"
        elif key == "JMS": 
            if shift == "up"   : jec_shifts[f"{key}_{shift}"] = "Mj_jmsUp_a"
            if shift == "down" : jec_shifts[f"{key}_{shift}"] = "Mj_jmsDown_a"
        elif key == "JMR": 
            if shift == "up"   : jec_shifts[f"{key}_{shift}"] = "Mj_jmrUp_a"
            if shift == "down" : jec_shifts[f"{key}_{shift}"] = "Mj_jmrDown_a"
        else: #split JES
            if shift == "up"   : jec_shifts[f"{key}_{shift}"] = f"Mj_jes{key}Up_a"
            if shift == "down" : jec_shifts[f"{key}_{shift}"] = f"Mj_jes{key}Down_a"

# ue_shifts = {}
# for key in uncluste:
#     for shift in ["up", "down"]:
#         if shift == "up"   : ue_shifts[f"{key}_{shift}"] = "MH_Reco_UE_up"
#         if shift == "down" : ue_shifts[f"{key}_{shift}"] = "MH_Reco_UE_down"
        
# pdf_shifts = {}
# #store nominal value first, and then the up/down variation
# for key in pdf:
#     for shift in ["up", "down"]:
#         if shift == "up"   : pdf_shifts[f"{key}_{shift}"] = "Mj_corr_V2_a"
#         if shift == "down" : pdf_shifts[f"{key}_{shift}"] = "Mj_corr_V2_a"

In [26]:
jec_shifts

{'JES_up': 'Mj_jesTotalUp_a',
 'JES_down': 'Mj_jesTotalDown_a',
 'JER_up': 'Mj_jerUp_a',
 'JER_down': 'Mj_jerDown_a',
 'JMS_up': 'Mj_jmsUp_a',
 'JMS_down': 'Mj_jmsDown_a',
 'JMR_up': 'Mj_jmrUp_a',
 'JMR_down': 'Mj_jmrDown_a',
 'Absolute_up': 'Mj_jesAbsoluteUp_a',
 'Absolute_down': 'Mj_jesAbsoluteDown_a',
 'Absolute_year_up': 'Mj_jesAbsolute_yearUp_a',
 'Absolute_year_down': 'Mj_jesAbsolute_yearDown_a',
 'BBEC1_up': 'Mj_jesBBEC1Up_a',
 'BBEC1_down': 'Mj_jesBBEC1Down_a',
 'BBEC1_year_up': 'Mj_jesBBEC1_yearUp_a',
 'BBEC1_year_down': 'Mj_jesBBEC1_yearDown_a',
 'EC2_up': 'Mj_jesEC2Up_a',
 'EC2_down': 'Mj_jesEC2Down_a',
 'EC2_year_up': 'Mj_jesEC2_yearUp_a',
 'EC2_year_down': 'Mj_jesEC2_yearDown_a',
 'FlavorQCD_up': 'Mj_jesFlavorQCDUp_a',
 'FlavorQCD_down': 'Mj_jesFlavorQCDDown_a',
 'HF_up': 'Mj_jesHFUp_a',
 'HF_down': 'Mj_jesHFDown_a',
 'HF_year_up': 'Mj_jesHF_yearUp_a',
 'HF_year_down': 'Mj_jesHF_yearDown_a',
 'RelativeBal_up': 'Mj_jesRelativeBalUp_a',
 'RelativeBal_down': 'Mj_jesRelativeBa

### Define CUT(aka. regions)

In [27]:
CUT = {
    "SR1" : {k:  (files[k]["nb_t_deep_ex"] == 0) & (files[k]["HbcVSQCS"] >= 0.998) for k in files},
    "SR2" : {k:  (files[k]["nb_t_deep_ex"] == 0) & (files[k]["HbcVSQCS"] >= 0.98) & (files[k]["HbcVSQCS"] <= 0.998) for k in files},
    "SR3" : {k:  (files[k]["nb_t_deep_ex"] == 0) & (files[k]["HbcVSQCS"] >= 0.9) & (files[k]["HbcVSQCS"] <= 0.98) for k in files},
    "CR"  : {k:  (files[k]["nb_t_deep_ex"] == 0) & (files[k]["HbcVSQCS"] < 0.9)  for k in files},
    "PS"  : {k:  (files[k]["nb_t_deep_ex"] >= 0)  for k in files},
}
# CR: tagger < 0.9

In [28]:
# files_cut = {typefile : {region : {} for region in CUT.keys()} for typefile in CustNanoData if (typefile != "QCD" and typefile != "data")}
# for typefile in files_cut:
#     if typefile != "PLACE_HOLDER":continue
#     print("file",typefile)
#     for region in CUT.keys():
#         print("region",region)
#         # files[typefile] = uproot.lazy({CustNanoData[typefile]: "PKUTree" })
#         files_cut[typefile] = files[typefile][CUT[region][typefile]]
#         if region.startswith("SR"):
#             continue
#             keep_keys = {
#                 "MH_Reco" : files_cut[typefile]["MH_Reco"][:1000000],
#                 "weight"  : files_cut[typefile]["weight"][:1000000],
#                 "LHEPdfWeight" : files_cut[typefile]["LHEPdfWeight"][:1000000],
#             }
#         else:
#             keep_keys = {
#                 "MH_Reco" : files_cut[typefile]["MH_Reco"][::50],
#                 "weight"  : files_cut[typefile]["weight"][::50],
#                 "LHEPdfWeight" : files_cut[typefile]["LHEPdfWeight"][::50],
#             }
        
#         # array_dict = {name : files_cut[typefile].array(library="ak") for name in keep_keys}
#         tmp_file_name = os.path.normpath(f"./root/{typefile}_{region}.root")
#         with uproot.recreate(tmp_file_name) as newfile:
#             newfile["PKUTree"] = keep_keys

### Define PDF variation individually

In [29]:
def get_pdf_unc(events,x_min = 50, x_max = 250, nbins = 20, up_down = "up", max_run = 100000, extra_weight = 1.0):
    '''
    input: events
    output: up, down variation histogram(both value and variance)
    get pdf variation and save it as a weight unc in the end
    since the computation is too large so we use a max_run cut
    '''
    histogram = np.zeros((len(events["LHEPdfWeight"][0]), nbins))
    num_weights = len(events["LHEPdfWeight"][0])
    # num_weights = 20
    hist_pdf_i = bh.Histogram(bh.axis.Regular(nbins, x_min, x_max), storage=bh.storage.Weight())
    # events_tmp = ak.copy(events)
    # Loop over each weight
    for i in range(0, num_weights):
        print("now pdf",i)
        print(len(events["LHEPdfWeight"]))
        hist_pdf_i.reset()
        hist_pdf_i.fill(events["MH_Reco"][0:max_run],weight=events["weight"][0:max_run] * events["LHEPdfWeight"][0:max_run][:, i] * extra_weight)
        histogram[i, :] = hist_pdf_i.view(flow=False).value
    # Calculate histogram errors
    histogram_stderr = np.sqrt(np.sum((histogram[1:] - histogram[0])**2, axis=0)) 
    print("nominal value:",histogram[0])
    # Calculate up and down variations
    if up_down == "up":
        variation_hist = histogram[0] + histogram_stderr
        print("up",variation_hist)
    elif up_down == "down":
        variation_hist = histogram[0] - histogram_stderr
        print("down",variation_hist)
    else: 
        raise ValueError("wrong variation")
    # print(variation_hist)
    return variation_hist

In [30]:
for k in files:
    # note that QCD and Data don't have such variation
    if k == "data" or k == "QCD": continue
    if k != "TT" : continue
    print("Add pdf of:",k)
    print(files[k])
    # get_pdf_unc(events=files[k][CUT["SR1a"][k]])
    # get_pdf_unc(events=uproot.lazy({f"./root/{year}_{k}_CR1.root": "PKUTree" }))
    # get_pdf_unc(events = uproot.lazy({f"./root/{k}_CR1.root": "PKUTree" }),up_down = "up", extra_weight = 1.0, max_run = 100000)
    # get_pdf_unc(events=uproot.lazy({f"./root/{k}_SR1b.root": "PKUTree" }))
    # get_pdf_unc(events=uproot.lazy({f"./root/{k}_SR2a.root": "PKUTree" }))
    # get_pdf_unc(events=uproot.lazy({f"./root/{k}_SR2b.root": "PKUTree" }))
    # get_pdf_unc(events=uproot.lazy({f"./root/{k}_CR1.root": "PKUTree" }))
    # get_pdf_unc(events=uproot.lazy({f"./root/{k}_CR2.root": "PKUTree" }))

### Save hist templates to pkl files

We note here that no particular operation is needed for QCD, since we only need raw QCD MC ratio as initial tranfer factor in the actual QCD prediction, and rhalphabet method will use (data - other bkg) in fail(control) region and perform simultaneous fit with pass(signal) region

In [31]:
def save_pkl(files, path_str = plot_dir, template_file = "templates",year_to_run = "2018"):
    
    templates = {} #empty dict to store the templates file
    regions = ["SR1","SR2","SR3","CR"] #signal regions or control regions
    years = ["2016APV", "2016", "2017", "2018"]
    samples = list(['data','QCD','Top','WJets','Rest','Signal']) #all samples we considered
    print("now running year:",year_to_run)
    #initialize weight based variation samples
    hist_samples = []
    for shift in ["down", "up"]:
        for wshift, wsyst in weight_shifts.items():
            for wsample in wsyst.samples:
                hist_samples.append(f"{wsample}_{wshift}_{shift}")
    
    hist_samples += samples
    #fill templates for different regions
    
    for region in regions:
        print("now processing:",region)
        templates[region] = hist2.Hist(
            hist2.axis.StrCategory(hist_samples, name="Sample"),
            *[shape_var.axis for shape_var in shape_vars],
            storage="weight",
            ) #initialize a hist object
        
        #add center value templates first
        for sample in samples:
            
            #if region is signal region and sample is signal samples, we multiply by lund plane sf
            weight_to_add = 1.0
            
            data = files[sample][CUT[region][sample]]
            templates[region].fill(
                                Sample=sample,
                                MH_Reco=data["Mj_corr_V2_a"],
                                weight=data["weight"] * weight_to_add,
                            )
            if sample == "data": 
                if (region.startswith("SR")):
                    # blind signal mass windows in pass region in data even for not "Blinded" region
                    # print("blind data of ",region)
                    for i, shape_var in enumerate(shape_vars):
                        if shape_var.blind_window is not None:
                            blindBins(templates[region], shape_var.blind_window, "data", axis=i)
                            


        #add weight based variation for each sample            
        for shift in ["down", "up"]:
            for wshift, wsyst in weight_shifts.items():
                for wsample in wsyst.samples:
                    if wsample in samples:
                        data = files[wsample][CUT[region][wsample]]
                        print(wsample, hex(id(files[wsample])))
                        weight_to_add = 1.0
                        templates[region].fill(
                            Sample=wsample + f"_{wshift}_{shift}",
                            MH_Reco=data["Mj_corr_V2_a"],
                            weight=data[f"{wshift}_{shift}"] * weight_to_add,
                        )
                        
        #add shift variation for each sample
        #1.initialize hist info
        for wshift, wsyst in jec_shifts.items():
            # split the JES/JER uncertainties according to year, i.e., one variation for each era
            templates[f"{region}_{wshift}"] = hist2.Hist(
            hist2.axis.StrCategory(samples, name="Sample"),
            *[shape_var.axis for shape_var in shape_vars],
            storage="weight",
            ) #initialize a hist object
        # for wshift, wsyst in pdf_shifts.items():
        #     templates[f"{region}_{wshift}"] = hist2.Hist(
        #     hist2.axis.StrCategory(samples, name="Sample"),
        #     *[shape_var.axis for shape_var in shape_vars],
        #     storage="weight",
        #     ) #initialize a hist object                
        #2.fill the hist
        for sample in mc_keys:
            data = files[sample][CUT[region][sample]] 
            #JECS
            for wshift, wsyst in jec_shifts.items():
                weight_to_add = 1.0
                #assign same variation as center value for other years
                templates[f"{region}_{wshift}"].fill(
                        Sample=sample,
                        MH_Reco=data[wsyst],
                        weight=data["weight"] * weight_to_add,
                    )                                    
            #pdf variation
            # for wshift, wsyst in pdf_shifts.items():
            #     weight_to_add = 1.0
            #     templates[f"{region}_{wshift}"].fill(
            #                     Sample=sample,pileup_down
            #                     MH_Reco=data[wsyst], #just nominal MET recovery mass
            #                     weight=data["weight"] * weight_to_add,
            #     )
            #     #but we need to change the up/down variation
            #     key_index = np.where(np.array(list(templates[f"{region}_{wshift}"].axes[0])) == sample)[0][0]
            #     print("now processing pdf variation", sample)
            #     #make following faster
            #     # data_tmp = ak.to_pandas(data)
            #     if "up" in wshift:
            #         #only apply change to SR but not CR!
            #         if region.startswith("SR"):
            #         # get_pdf_unc(events = data_tmp,up_down = "up", extra_weight = weight_to_add, max_run = 100000)
            #             templates[f"{region}_{wshift}"].view(flow = False)[key_index][:].value = get_pdf_unc(events = uproot.lazy({f"./root/{year}_{sample}_{region}.root": "PKUTree" }),up_down = "up", extra_weight = weight_to_add, max_run = 100000)
            #     elif "down" in wshift:
            #         if region.startswith("SR"):
            #         # get_pdf_unc(events = data_tmp,up_down = "down", extra_weight = weight_to_add, max_run = 100000)
            #             templates[f"{region}_{wshift}"].view(flow = False)[key_index][:].value = get_pdf_unc(events = uproot.lazy({f"./root/{year}_{sample}_{region}.root": "PKUTree" }),up_down = "down", extra_weight = weight_to_add, max_run = 100000)
                # print("now done processing pdf variation",sample)
        
        #extra process for QCD and data, is this really needed?
        # for sample in ["QCD","data"]: # actually no need to run QCD as not used at all
        # for sample in ["data"]:
        #     #QCD and data doesn't have any j-shift nor weight based variation
        #     print("now processing(QCD/data):",sample)
        #     #JEC variation
        #     for wshift, wsyst in jec_shifts.items():
        #         data = files[sample][CUT[region][sample]] 
        #         #always assign value with `MH_Reco` variable
        #         templates[f"{region}_{wshift}"].fill(
        #                 Sample=sample,
        #                 MH_Reco=data["Mj_corr_V2_a"],
        #                 weight=data["weight"],
        #             )
                
        #         #do blind procedure
        #         if sample == "data" and  (region.startswith("SR")):
        #             for i, shape_var in enumerate(shape_vars):
        #                 if shape_var.blind_window is not None:
        #                     blindBins(templates[f"{region}_{wshift}"], shape_var.blind_window, "data", axis=i)

            #pdf variation
            # for wshift, wsyst in pdf_shifts.items():
            #     data = files[sample][CUT[region][sample]] 
            #     #always assign value with `MH_Reco` variable
            #     templates[f"{region}_{wshift}"].fill(
            #             Sample=sample,
            #             MH_Reco=data["MH_Reco"],
            #             weight=data["weight"],
            #         )           
            #     #do blind procedure
            #     if sample == "data" and (region.endswith("a") or region.endswith("b")):
            #         for i, shape_var in enumerate(shape_vars):
            #             if shape_var.blind_window is not None:
            #                 blindBins(templates[f"{region}_{wshift}"], shape_var.blind_window, "data", axis=i)
                        
        print("done fill template ",region)        
    
    #Creates blinded copies of each region's templates and saves a pickle of the templates
    blind_window = shape_vars[0].blind_window
    for label, template in list(templates.items()):
        blinded_template = deepcopy(template)
        blindBins(blinded_template, blind_window)
        templates[f"{label}Blinded"] = blinded_template
    
    #save files
    with open(f"{path_str}/{template_file}_{year_to_run}.pkl", "wb") as fp:
        pkl.dump(templates, fp) # dump the templates of each region in a pkl file
        print("Saved templates to", f"{template_file}_{year_to_run}.pkl")

In [ ]:
save_pkl(files = files, year_to_run = YEAR)

now running year: 2018
now processing: SR1
Signal 0x7f13c8f91a30
Top 0x7f15cd9e2a30
WJets 0x7f15ce252520
Rest 0x7f1d75ea4250
Signal 0x7f13c8f91a30
Top 0x7f15cd9e2a30
WJets 0x7f15ce252520
Rest 0x7f1d75ea4250
Signal 0x7f13c8f91a30
Top 0x7f15cd9e2a30
WJets 0x7f15ce252520
Rest 0x7f1d75ea4250
Top 0x7f15cd9e2a30
WJets 0x7f15ce252520
Rest 0x7f1d75ea4250
Signal 0x7f13c8f91a30
Top 0x7f15cd9e2a30
WJets 0x7f15ce252520
Rest 0x7f1d75ea4250
Signal 0x7f13c8f91a30
Top 0x7f15cd9e2a30
WJets 0x7f15ce252520
Rest 0x7f1d75ea4250
Signal 0x7f13c8f91a30
Top 0x7f15cd9e2a30
WJets 0x7f15ce252520
Rest 0x7f1d75ea4250
Signal 0x7f13c8f91a30
Top 0x7f15cd9e2a30
WJets 0x7f15ce252520
Rest 0x7f1d75ea4250
Signal 0x7f13c8f91a30
Top 0x7f15cd9e2a30
WJets 0x7f15ce252520
Rest 0x7f1d75ea4250
Top 0x7f15cd9e2a30
WJets 0x7f15ce252520
Rest 0x7f1d75ea4250
Signal 0x7f13c8f91a30
Top 0x7f15cd9e2a30
WJets 0x7f15ce252520
Rest 0x7f1d75ea4250
Signal 0x7f13c8f91a30
Top 0x7f15cd9e2a30
WJets 0x7f15ce252520
Rest 0x7f1d75ea4250
done fill templat

In [ ]:
PLACE_HOLDER

### test about the templates

In [ ]:
with open(f"{plot_dir}/templates_2018.pkl","rb") as f:
    hists_template1 = pkl.load(f)
hists_template1["SR1a"]["ggF",:]

In [ ]:
hists_template1["SR2a"]["ggF_pileup_up",:]

In [ ]:
hists_template1["SR2a"]["ggF",:]

In [ ]:
hists_template1["SR2a"]["ggF_pileup_down",:]

In [ ]:
hists_template1.keys()

In [ ]:
hists_template1["SR1a"]

In [ ]:
hists_template1["SR1a_JES_up_2018"]

In [ ]:
hists_template1["SR1a"]["TT_FSRPartonShower_up",:]

In [ ]:
hists_template1["SR1a"]["TT",:]

In [ ]:
hists_template1["SR1a"]["TT_FSRPartonShower_down",:]

In [ ]:
sample_template1 = hists_template1["SR1a"]["QCD", :]
err = sample_template1.variances()
err

In [ ]:
for i , axis in enumerate(hists_template1["SR1a"].axes[1:]):
    print(i, axis)

### Some test of 1l side analysis for reference

In [ ]:
# with open("/home/pku/zhaoyz/Higgs/HHbbVV/src/HHbbVV/postprocessing/templates/23Jun14/2018_templates.pkl","rb") as f:
with open("/ospool/cms-user/yuzhe/BoostedHWW/prediction/boostedhiggs/combine/templates/v4/hists_templates_Run2.pkl","rb") as f:
    hists_template2 = pkl.load(f)
# hists_template["pass"]["QCD",:]  
# hists_template["pass"]["QCD",:].sum().value


In [ ]:
hists_template2

### Some test of HHbbVV analysis for reference

In [ ]:
# with open("/home/pku/zhaoyz/Higgs/HHbbVV/src/HHbbVV/postprocessing/templates/23Jun14/2018_templates.pkl","rb") as f:
with open("/ospool/cms-user/yuzhe/BoostedHWW/prediction/HHbbVV/src/HHbbVV/postprocessing/templates/24Mar15UpdateData/2018_templates.pkl","rb") as f:
    hists_template2 = pkl.load(f)
# hists_template["pass"]["QCD",:]  
# hists_template["pass"]["QCD",:].sum().value


In [ ]:
hists_template2.keys()

In [ ]:
hists_template2["pass"]

In [ ]:
hists_template2["pass_JES_up"]["ST",:]

In [ ]:
Hist(
  StrCategory(['HHbbVV', 'ggHH_kl_2p45_kt_1_HHbbVV', 'ggHH_kl_5_kt_1_HHbbVV', 'ggHH_kl_0_kt_1_HHbbVV', 'VBFHHbbVV', 'qqHH_CV_1_C2V_0_kl_1_HHbbVV', 'qqHH_CV_1p5_C2V_1_kl_1_HHbbVV', 'qqHH_CV_1_C2V_1_kl_2_HHbbVV', 'qqHH_CV_1_C2V_2_kl_1_HHbbVV', 'qqHH_CV_1_C2V_1_kl_0_HHbbVV', 'QCD', 'Top', 'W+Jets', 'Z+Jets', 'Diboson', 'ggFHbb', 'VBFHbb', 'ZHbb', 'WHbb', 'ggZHbb', 'ttHbb', 'HWW', 'Data', 'HHbbVV_txbb_down', 'ggHH_kl_2p45_kt_1_HHbbVV_txbb_down', 'ggHH_kl_5_kt_1_HHbbVV_txbb_down', 'ggHH_kl_0_kt_1_HHbbVV_txbb_down', 'VBFHHbbVV_txbb_down', 'qqHH_CV_1_C2V_0_kl_1_HHbbVV_txbb_down', 'qqHH_CV_1p5_C2V_1_kl_1_HHbbVV_txbb_down', 'qqHH_CV_1_C2V_1_kl_2_HHbbVV_txbb_down', 'qqHH_CV_1_C2V_2_kl_1_HHbbVV_txbb_down', 'qqHH_CV_1_C2V_1_kl_0_HHbbVV_txbb_down', 'HHbbVV_pileup_down', 'ggHH_kl_2p45_kt_1_HHbbVV_pileup_down', 'ggHH_kl_5_kt_1_HHbbVV_pileup_down', 'ggHH_kl_0_kt_1_HHbbVV_pileup_down', 'VBFHHbbVV_pileup_down', 'qqHH_CV_1_C2V_0_kl_1_HHbbVV_pileup_down', 'qqHH_CV_1p5_C2V_1_kl_1_HHbbVV_pileup_down', 'qqHH_CV_1_C2V_1_kl_2_HHbbVV_pileup_down', 'qqHH_CV_1_C2V_2_kl_1_HHbbVV_pileup_down', 'qqHH_CV_1_C2V_1_kl_0_HHbbVV_pileup_down', 'TT_pileup_down', 'ST_pileup_down', 'W+Jets_pileup_down', 'Z+Jets_pileup_down', 'HHbbVV_pileupID_down', 'ggHH_kl_2p45_kt_1_HHbbVV_pileupID_down', 'ggHH_kl_5_kt_1_HHbbVV_pileupID_down', 'ggHH_kl_0_kt_1_HHbbVV_pileupID_down', 'VBFHHbbVV_pileupID_down', 'qqHH_CV_1_C2V_0_kl_1_HHbbVV_pileupID_down', 'qqHH_CV_1p5_C2V_1_kl_1_HHbbVV_pileupID_down', 'qqHH_CV_1_C2V_1_kl_2_HHbbVV_pileupID_down', 'qqHH_CV_1_C2V_2_kl_1_HHbbVV_pileupID_down', 'qqHH_CV_1_C2V_1_kl_0_HHbbVV_pileupID_down', 'TT_pileupID_down', 'ST_pileupID_down', 'W+Jets_pileupID_down', 'Z+Jets_pileupID_down', 'HHbbVV_ISRPartonShower_down', 'ggHH_kl_2p45_kt_1_HHbbVV_ISRPartonShower_down', 'ggHH_kl_5_kt_1_HHbbVV_ISRPartonShower_down', 'ggHH_kl_0_kt_1_HHbbVV_ISRPartonShower_down', 'VBFHHbbVV_ISRPartonShower_down', 'qqHH_CV_1_C2V_0_kl_1_HHbbVV_ISRPartonShower_down', 'qqHH_CV_1p5_C2V_1_kl_1_HHbbVV_ISRPartonShower_down', 'qqHH_CV_1_C2V_1_kl_2_HHbbVV_ISRPartonShower_down', 'qqHH_CV_1_C2V_2_kl_1_HHbbVV_ISRPartonShower_down', 'qqHH_CV_1_C2V_1_kl_0_HHbbVV_ISRPartonShower_down', 'TT_ISRPartonShower_down', 'ST_ISRPartonShower_down', 'W+Jets_ISRPartonShower_down', 'Z+Jets_ISRPartonShower_down', 'HHbbVV_FSRPartonShower_down', 'ggHH_kl_2p45_kt_1_HHbbVV_FSRPartonShower_down', 'ggHH_kl_5_kt_1_HHbbVV_FSRPartonShower_down', 'ggHH_kl_0_kt_1_HHbbVV_FSRPartonShower_down', 'VBFHHbbVV_FSRPartonShower_down', 'qqHH_CV_1_C2V_0_kl_1_HHbbVV_FSRPartonShower_down', 'qqHH_CV_1p5_C2V_1_kl_1_HHbbVV_FSRPartonShower_down', 'qqHH_CV_1_C2V_1_kl_2_HHbbVV_FSRPartonShower_down', 'qqHH_CV_1_C2V_2_kl_1_HHbbVV_FSRPartonShower_down', 'qqHH_CV_1_C2V_1_kl_0_HHbbVV_FSRPartonShower_down', 'TT_FSRPartonShower_down', 'ST_FSRPartonShower_down', 'W+Jets_FSRPartonShower_down', 'Z+Jets_FSRPartonShower_down', 'HHbbVV_L1EcalPrefiring_down', 'ggHH_kl_2p45_kt_1_HHbbVV_L1EcalPrefiring_down', 'ggHH_kl_5_kt_1_HHbbVV_L1EcalPrefiring_down', 'ggHH_kl_0_kt_1_HHbbVV_L1EcalPrefiring_down', 'VBFHHbbVV_L1EcalPrefiring_down', 'qqHH_CV_1_C2V_0_kl_1_HHbbVV_L1EcalPrefiring_down', 'qqHH_CV_1p5_C2V_1_kl_1_HHbbVV_L1EcalPrefiring_down', 'qqHH_CV_1_C2V_1_kl_2_HHbbVV_L1EcalPrefiring_down', 'qqHH_CV_1_C2V_2_kl_1_HHbbVV_L1EcalPrefiring_down', 'qqHH_CV_1_C2V_1_kl_0_HHbbVV_L1EcalPrefiring_down', 'TT_L1EcalPrefiring_down', 'ST_L1EcalPrefiring_down', 'W+Jets_L1EcalPrefiring_down', 'Z+Jets_L1EcalPrefiring_down', 'HHbbVV_electron_id_down', 'ggHH_kl_2p45_kt_1_HHbbVV_electron_id_down', 'ggHH_kl_5_kt_1_HHbbVV_electron_id_down', 'ggHH_kl_0_kt_1_HHbbVV_electron_id_down', 'VBFHHbbVV_electron_id_down', 'qqHH_CV_1_C2V_0_kl_1_HHbbVV_electron_id_down', 'qqHH_CV_1p5_C2V_1_kl_1_HHbbVV_electron_id_down', 'qqHH_CV_1_C2V_1_kl_2_HHbbVV_electron_id_down', 'qqHH_CV_1_C2V_2_kl_1_HHbbVV_electron_id_down', 'qqHH_CV_1_C2V_1_kl_0_HHbbVV_electron_id_down', 'TT_electron_id_down', 'ST_electron_id_down', 'W+Jets_electron_id_down', 'Z+Jets_electron_id_down', 'HHbbVV_muon_id_down', 'ggHH_kl_2p45_kt_1_HHbbVV_muon_id_down', 'ggHH_kl_5_kt_1_HHbbVV_muon_id_down', 'ggHH_kl_0_kt_1_HHbbVV_muon_id_down', 'VBFHHbbVV_muon_id_down', 'qqHH_CV_1_C2V_0_kl_1_HHbbVV_muon_id_down', 'qqHH_CV_1p5_C2V_1_kl_1_HHbbVV_muon_id_down', 'qqHH_CV_1_C2V_1_kl_2_HHbbVV_muon_id_down', 'qqHH_CV_1_C2V_2_kl_1_HHbbVV_muon_id_down', 'qqHH_CV_1_C2V_1_kl_0_HHbbVV_muon_id_down', 'TT_muon_id_down', 'ST_muon_id_down', 'W+Jets_muon_id_down', 'Z+Jets_muon_id_down', 'HHbbVV_scale_down', 'ggHH_kl_2p45_kt_1_HHbbVV_scale_down', 'ggHH_kl_5_kt_1_HHbbVV_scale_down', 'ggHH_kl_0_kt_1_HHbbVV_scale_down', 'VBFHHbbVV_scale_down', 'qqHH_CV_1_C2V_0_kl_1_HHbbVV_scale_down', 'qqHH_CV_1p5_C2V_1_kl_1_HHbbVV_scale_down', 'qqHH_CV_1_C2V_1_kl_2_HHbbVV_scale_down', 'qqHH_CV_1_C2V_2_kl_1_HHbbVV_scale_down', 'qqHH_CV_1_C2V_1_kl_0_HHbbVV_scale_down', 'TT_scale_down', 'HHbbVV_pdf_down', 'ggHH_kl_2p45_kt_1_HHbbVV_pdf_down', 'ggHH_kl_5_kt_1_HHbbVV_pdf_down', 'ggHH_kl_0_kt_1_HHbbVV_pdf_down', 'VBFHHbbVV_pdf_down', 'qqHH_CV_1_C2V_0_kl_1_HHbbVV_pdf_down', 'qqHH_CV_1p5_C2V_1_kl_1_HHbbVV_pdf_down', 'qqHH_CV_1_C2V_1_kl_2_HHbbVV_pdf_down', 'qqHH_CV_1_C2V_2_kl_1_HHbbVV_pdf_down', 'qqHH_CV_1_C2V_1_kl_0_HHbbVV_pdf_down', 'HHbbVV_txbb_up', 'ggHH_kl_2p45_kt_1_HHbbVV_txbb_up', 'ggHH_kl_5_kt_1_HHbbVV_txbb_up', 'ggHH_kl_0_kt_1_HHbbVV_txbb_up', 'VBFHHbbVV_txbb_up', 'qqHH_CV_1_C2V_0_kl_1_HHbbVV_txbb_up', 'qqHH_CV_1p5_C2V_1_kl_1_HHbbVV_txbb_up', 'qqHH_CV_1_C2V_1_kl_2_HHbbVV_txbb_up', 'qqHH_CV_1_C2V_2_kl_1_HHbbVV_txbb_up', 'qqHH_CV_1_C2V_1_kl_0_HHbbVV_txbb_up', 'HHbbVV_pileup_up', 'ggHH_kl_2p45_kt_1_HHbbVV_pileup_up', 'ggHH_kl_5_kt_1_HHbbVV_pileup_up', 'ggHH_kl_0_kt_1_HHbbVV_pileup_up', 'VBFHHbbVV_pileup_up', 'qqHH_CV_1_C2V_0_kl_1_HHbbVV_pileup_up', 'qqHH_CV_1p5_C2V_1_kl_1_HHbbVV_pileup_up', 'qqHH_CV_1_C2V_1_kl_2_HHbbVV_pileup_up', 'qqHH_CV_1_C2V_2_kl_1_HHbbVV_pileup_up', 'qqHH_CV_1_C2V_1_kl_0_HHbbVV_pileup_up', 'TT_pileup_up', 'ST_pileup_up', 'W+Jets_pileup_up', 'Z+Jets_pileup_up', 'HHbbVV_pileupID_up', 'ggHH_kl_2p45_kt_1_HHbbVV_pileupID_up', 'ggHH_kl_5_kt_1_HHbbVV_pileupID_up', 'ggHH_kl_0_kt_1_HHbbVV_pileupID_up', 'VBFHHbbVV_pileupID_up', 'qqHH_CV_1_C2V_0_kl_1_HHbbVV_pileupID_up', 'qqHH_CV_1p5_C2V_1_kl_1_HHbbVV_pileupID_up', 'qqHH_CV_1_C2V_1_kl_2_HHbbVV_pileupID_up', 'qqHH_CV_1_C2V_2_kl_1_HHbbVV_pileupID_up', 'qqHH_CV_1_C2V_1_kl_0_HHbbVV_pileupID_up', 'TT_pileupID_up', 'ST_pileupID_up', 'W+Jets_pileupID_up', 'Z+Jets_pileupID_up', 'HHbbVV_ISRPartonShower_up', 'ggHH_kl_2p45_kt_1_HHbbVV_ISRPartonShower_up', 'ggHH_kl_5_kt_1_HHbbVV_ISRPartonShower_up', 'ggHH_kl_0_kt_1_HHbbVV_ISRPartonShower_up', 'VBFHHbbVV_ISRPartonShower_up', 'qqHH_CV_1_C2V_0_kl_1_HHbbVV_ISRPartonShower_up', 'qqHH_CV_1p5_C2V_1_kl_1_HHbbVV_ISRPartonShower_up', 'qqHH_CV_1_C2V_1_kl_2_HHbbVV_ISRPartonShower_up', 'qqHH_CV_1_C2V_2_kl_1_HHbbVV_ISRPartonShower_up', 'qqHH_CV_1_C2V_1_kl_0_HHbbVV_ISRPartonShower_up', 'TT_ISRPartonShower_up', 'ST_ISRPartonShower_up', 'W+Jets_ISRPartonShower_up', 'Z+Jets_ISRPartonShower_up', 'HHbbVV_FSRPartonShower_up', 'ggHH_kl_2p45_kt_1_HHbbVV_FSRPartonShower_up', 'ggHH_kl_5_kt_1_HHbbVV_FSRPartonShower_up', 'ggHH_kl_0_kt_1_HHbbVV_FSRPartonShower_up', 'VBFHHbbVV_FSRPartonShower_up', 'qqHH_CV_1_C2V_0_kl_1_HHbbVV_FSRPartonShower_up', 'qqHH_CV_1p5_C2V_1_kl_1_HHbbVV_FSRPartonShower_up', 'qqHH_CV_1_C2V_1_kl_2_HHbbVV_FSRPartonShower_up', 'qqHH_CV_1_C2V_2_kl_1_HHbbVV_FSRPartonShower_up', 'qqHH_CV_1_C2V_1_kl_0_HHbbVV_FSRPartonShower_up', 'TT_FSRPartonShower_up', 'ST_FSRPartonShower_up', 'W+Jets_FSRPartonShower_up', 'Z+Jets_FSRPartonShower_up', 'HHbbVV_L1EcalPrefiring_up', 'ggHH_kl_2p45_kt_1_HHbbVV_L1EcalPrefiring_up', 'ggHH_kl_5_kt_1_HHbbVV_L1EcalPrefiring_up', 'ggHH_kl_0_kt_1_HHbbVV_L1EcalPrefiring_up', 'VBFHHbbVV_L1EcalPrefiring_up', 'qqHH_CV_1_C2V_0_kl_1_HHbbVV_L1EcalPrefiring_up', 'qqHH_CV_1p5_C2V_1_kl_1_HHbbVV_L1EcalPrefiring_up', 'qqHH_CV_1_C2V_1_kl_2_HHbbVV_L1EcalPrefiring_up', 'qqHH_CV_1_C2V_2_kl_1_HHbbVV_L1EcalPrefiring_up', 'qqHH_CV_1_C2V_1_kl_0_HHbbVV_L1EcalPrefiring_up', 'TT_L1EcalPrefiring_up', 'ST_L1EcalPrefiring_up', 'W+Jets_L1EcalPrefiring_up', 'Z+Jets_L1EcalPrefiring_up', 'HHbbVV_electron_id_up', 'ggHH_kl_2p45_kt_1_HHbbVV_electron_id_up', 'ggHH_kl_5_kt_1_HHbbVV_electron_id_up', 'ggHH_kl_0_kt_1_HHbbVV_electron_id_up', 'VBFHHbbVV_electron_id_up', 'qqHH_CV_1_C2V_0_kl_1_HHbbVV_electron_id_up', 'qqHH_CV_1p5_C2V_1_kl_1_HHbbVV_electron_id_up', 'qqHH_CV_1_C2V_1_kl_2_HHbbVV_electron_id_up', 'qqHH_CV_1_C2V_2_kl_1_HHbbVV_electron_id_up', 'qqHH_CV_1_C2V_1_kl_0_HHbbVV_electron_id_up', 'TT_electron_id_up', 'ST_electron_id_up', 'W+Jets_electron_id_up', 'Z+Jets_electron_id_up', 'HHbbVV_muon_id_up', 'ggHH_kl_2p45_kt_1_HHbbVV_muon_id_up', 'ggHH_kl_5_kt_1_HHbbVV_muon_id_up', 'ggHH_kl_0_kt_1_HHbbVV_muon_id_up', 'VBFHHbbVV_muon_id_up', 'qqHH_CV_1_C2V_0_kl_1_HHbbVV_muon_id_up', 'qqHH_CV_1p5_C2V_1_kl_1_HHbbVV_muon_id_up', 'qqHH_CV_1_C2V_1_kl_2_HHbbVV_muon_id_up', 'qqHH_CV_1_C2V_2_kl_1_HHbbVV_muon_id_up', 'qqHH_CV_1_C2V_1_kl_0_HHbbVV_muon_id_up', 'TT_muon_id_up', 'ST_muon_id_up', 'W+Jets_muon_id_up', 'Z+Jets_muon_id_up', 'HHbbVV_scale_up', 'ggHH_kl_2p45_kt_1_HHbbVV_scale_up', 'ggHH_kl_5_kt_1_HHbbVV_scale_up', 'ggHH_kl_0_kt_1_HHbbVV_scale_up', 'VBFHHbbVV_scale_up', 'qqHH_CV_1_C2V_0_kl_1_HHbbVV_scale_up', 'qqHH_CV_1p5_C2V_1_kl_1_HHbbVV_scale_up', 'qqHH_CV_1_C2V_1_kl_2_HHbbVV_scale_up', 'qqHH_CV_1_C2V_2_kl_1_HHbbVV_scale_up', 'qqHH_CV_1_C2V_1_kl_0_HHbbVV_scale_up', 'TT_scale_up', 'HHbbVV_pdf_up', 'ggHH_kl_2p45_kt_1_HHbbVV_pdf_up', 'ggHH_kl_5_kt_1_HHbbVV_pdf_up', 'ggHH_kl_0_kt_1_HHbbVV_pdf_up', 'VBFHHbbVV_pdf_up', 'qqHH_CV_1_C2V_0_kl_1_HHbbVV_pdf_up', 'qqHH_CV_1p5_C2V_1_kl_1_HHbbVV_pdf_up', 'qqHH_CV_1_C2V_1_kl_2_HHbbVV_pdf_up', 'qqHH_CV_1_C2V_2_kl_1_HHbbVV_pdf_up', 'qqHH_CV_1_C2V_1_kl_0_HHbbVV_pdf_up'], name='Sample'),
  Regular(20, 50, 250, name='bbFatJetParticleNetMass', label='$m^{bb}_\\mathrm{Reg}$ (GeV)'),
  storage=Weight()) # Sum: WeightedSum(value=-nan, variance=88.7794) (WeightedSum(value=-nan, variance=88.7794) with flow)

In [ ]:
hists_template2["pass"]["ST",:]

In [ ]:
hists_template2["pass_JES_down"]["ST",:]

In [ ]:
hists_template2["pass"]["HHbbVV_pileup_up",:]

In [ ]:
hists_template2["pass"]["HHbbVV_pileup_up",:]

In [ ]:
hists_template2["pass"]["HHbbVV",:]

In [ ]:
hists_template2["passBlinded"]

### test how to load and use the *.pkl template file

In [ ]:
def get_template(h, sample):
    ''' 
    histogram h Hist, with axes:["samples","systematic","MH_Reco"]
    sample is sample name in ["QCD",...,"data"]
    '''
    mass_axis = 1 #axis index
    massbins = h.axes[mass_axis].edges
    return (h[sample, :].values(), h[sample, :].variances(), massbins, "MH_Reco")

a = get_template(hists_template1["SR1a"],"QCD")
a

### make some plots to test the variation

In [ ]:
nbins, x_min, x_max = 20, 50, 250

plt.rcParams['axes.prop_cycle'] = cycler(color=["tab:blue",	"tab:orange",	"tab:green",	"tab:red",	"tab:purple", "tab:brown", "tab:pink", "k","tab:olive" ,	"tab:cyan"])

f = plt.figure(figsize=(14, 15))
ax = f.add_subplot(1, 1, 1)  
ax.grid()

year = "2018"
LUMI = {"2016APV": 36.33,"2016": 36.33, "2017": 41.48, "2018": 59.83,"Full-Run2":138}
hep.cms.label(loc = 1, data=True, year=year, ax=ax, lumi=LUMI[year], fontsize=18, llabel='Preliminary')

hist_region = bh.Histogram(bh.axis.Regular(nbins, x_min, x_max), storage=bh.storage.Weight())
hep.histplot(get_template(hists_template1["CR1_pdfscale_up"],"TT")[0], bins=get_template(hists_template1["SR1a_JES_up"],"TT")[2], label="Test1 ", histtype='step', stack=False, linewidth=2, ax=ax,color = "red")

hist_region = bh.Histogram(bh.axis.Regular(nbins, x_min, x_max), storage=bh.storage.Weight())
hep.histplot(get_template(hists_template1["CR1"],"TT")[0], bins=get_template(hists_template1["SR1a_JES_up"],"TT")[2], label="Test2 ", histtype='step', stack=False, linewidth=2, ax=ax,color = "blue")

hist_region = bh.Histogram(bh.axis.Regular(nbins, x_min, x_max), storage=bh.storage.Weight())
hep.histplot(get_template(hists_template1["CR1_pdfscale_down"],"TT")[0], bins=get_template(hists_template1["SR1a_JES_up"],"TT")[2], label="Test3 ", histtype='step', stack=False, linewidth=2, ax=ax,color = "orange")


ax.set_ylabel("Events")
ax.legend(loc="upper right", ncol=1, frameon=False, fontsize=18)
plt.show()

### make plots for PDF weight more professionally

In [ ]:
MAIN_DIR = "."

plot_dir_pro = f"{MAIN_DIR}/plots/variation/8May24"
_ = os.system(f"mkdir -p {plot_dir_pro}")

region = "SR1a"
files_str = "Rest"
year = YEAR
bins = get_template(hists_template1[f"{region}"],"WJets")[2]

plt.rcParams['axes.prop_cycle'] = cycler(color=["tab:blue",	"tab:orange",	"tab:green",	"tab:red",	"tab:purple", "tab:brown", "tab:pink", "k","tab:olive" ,	"tab:cyan"])
# plt.figure(figsize=(14,10))
f = plt.figure(figsize=(14, 15))
gs = mpl.gridspec.GridSpec(2, 1, height_ratios=[3, 1], hspace=0.08)
ax = f.add_subplot(gs[0])
plt.grid()
LUMI = {"2016": 36.33, "2017": 41.48, "2018": 59.83,"Full-Run2":138}
hep.cms.label(loc = 1, data=True, year=year, ax=ax, lumi=LUMI[year], fontsize=18, llabel='Preliminary')
ax1 = f.add_subplot(gs[1])
ax1.grid()

hist_value_up = get_template(hists_template1[f"{region}_pdfscale_up"],files_str)[0]
hist_var_up   = get_template(hists_template1[f"{region}_pdfscale_up"],files_str)[1]
err_up = np.nan_to_num(error_bar(hist_value_up, hist_var_up, type = "mc"), nan = 0)
hep.histplot(hist_value_up, bins=bins, yerr=err_up, label="PDF scale up", histtype='step', stack=False, linewidth=2, ax=ax,color = "red")


hist_value_nom = get_template(hists_template1[f"{region}"],files_str)[0]
hist_var_nom   = get_template(hists_template1[f"{region}"],files_str)[1]
err_nom = np.nan_to_num(error_bar(hist_value_nom, hist_var_nom, type = "mc"), nan = 0)
hep.histplot(hist_value_nom, bins=bins, yerr=err_nom, label="center", histtype='step', stack=False, linewidth=2, ax=ax,color = "black")


hist_value_down = get_template(hists_template1[f"{region}_pdfscale_down"],files_str)[0]
hist_var_down   = get_template(hists_template1[f"{region}_pdfscale_down"],files_str)[1]
err_down = np.nan_to_num(error_bar(hist_value_down, hist_var_down, type = "mc"), nan = 0)
hep.histplot(hist_value_down, bins=bins, yerr=err_down, label="PDF scale down", histtype='step', stack=False, linewidth=2, ax=ax,color = "blue")

err_up   = np.sqrt(np.power(err_up/hist_value_up,2) + np.power(err_nom/hist_value_nom,2))
hep.histplot(hist_value_up/hist_value_nom,   bins=bins, yerr=err_up, color='red',  label="PDF scale up/center", histtype='step', density=False, stack=False, ax=ax1,linewidth=2)
err_down = np.sqrt(np.power(err_down/hist_value_down,2) + np.power(err_nom/hist_value_nom,2))    
hep.histplot(hist_value_down/hist_value_nom, bins=bins, yerr=err_down, color='blue', label="PDF scale down/center",histtype='step', density=False, stack=False, ax=ax1,linewidth=2)    


ax1.set_xlabel("Higgs candidate MET recovery jet mass(GeV)")
ax.set_ylabel("Events")
ax1.set_ylabel("ratio")
ax1.set_ylim(0, 2)
# ax.set_yscale('log') 
ax.legend(loc="upper right", ncol=1, frameon=False, fontsize=22)
ax1.legend(loc="upper right", ncol=1, frameon=False, fontsize=20)
plt.text(0.05,0.83,region+ ", " + files_str,fontsize=24, color="black", ha='left',transform=ax.transAxes)
plt.savefig(f"{plot_dir_pro}/pdf_{year}_{region}{files_str}.pdf", bbox_inches='tight')    

### Some other test

In [ ]:
regions = {
        "CR1" :{"SRa": "SR1a","SRb":"SR1b"},
        "CR2" :{"SRa": "SR2a","SRb":"SR2b"},
        "CR3" :{"SRa": "SR3a","SRb":"SR3b"},
        }

regions_blinded = { key_fail + "_blinded": {key_pass + "_blinded" : key_pass_ab + "_blinded" for key_pass , key_pass_ab in key_pass_dict.items()}  for key_fail , key_pass_dict in regions.items()}
regions_blinded.keys()

In [ ]:
region = "SR1a_blinded"
pass_region = ("a_" in region)
pass_region

In [ ]:
region = "SR1aBlinded"
region_noblinded = region.split("Blinded")[0]
region_noblinded

In [ ]:
for files in os.listdir("./root/"):
    if not files.startswith("2017"):
        os.system(f"mv ./root/{files} ./root/2018_{files}")